# MCP를 통한 External Database 활용

이 노트북에서는 Strands Agents와 MCP (Model Context Protocol)를 사용하여 외부 데이터베이스인 Arxiv와 ChEMBL을 연동하는 방법을 실습합니다.

## 학습 목표
- MCP를 통해 외부 데이터베이스를 Agent의 도구로 활용하는 방법 이해
- 기본 Agent와 MCP 활용 Agent의 성능 차이 비교
- 실제 연구 질문에 대한 답변 생성 실습

## 1. 환경 설정

필요한 라이브러리와 의존성을 설치합니다.

In [ ]:
# 필요한 패키지 설치
%pip install strands-agents strands-agents-tools mcp boto3 arxiv chembl-webresource-client python-dateutil --quiet

In [1]:
# 라이브러리 임포트
import sys
import logging
import asyncio
from typing import List, Dict, Any
from datetime import datetime

# AWS SDK
import boto3
from botocore.config import Config

# Strands Agents
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient

# MCP
from mcp import stdio_client, StdioServerParameters
from mcp.server.fastmcp import FastMCP

# External Libraries
import arxiv
from chembl_webresource_client.new_client import new_client

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("external_dbs_demo")

## 2. MCP Server 구현

### 2.1 Arxiv MCP Server

In [2]:
%%writefile mcp_server_arxiv_demo.py
from mcp.server.fastmcp import FastMCP
import arxiv
import logging
import sys
from typing import Dict, Any, List
from datetime import datetime, timezone
from dateutil import parser

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(filename)s:%(lineno)d | %(message)s',
    handlers=[
        logging.StreamHandler(sys.stderr)
    ]
)
logger = logging.getLogger("arxiv_mcp")

# Maximum number of results setting
MAX_RESULTS = 10

# FastMCP initialization
try:
    mcp = FastMCP(
        name="arxiv_tools",
    )
    logger.info("arXiv MCP server initialized successfully")
except Exception as e:
    err_msg = f"Error: {str(e)}"
    logger.error(f"{err_msg}")

def _is_within_date_range(
    date: datetime, start: datetime | None, end: datetime | None
) -> bool:
    """Check if a date falls within the specified range."""
    if start and not start.tzinfo:
        start = start.replace(tzinfo=timezone.utc)
    if end and not end.tzinfo:
        end = end.replace(tzinfo=timezone.utc)

    if start and date < start:
        return False
    if end and date > end:
        return False
    return True

def _process_paper(paper: arxiv.Result) -> Dict[str, Any]:
    """Process paper information with resource URI."""
    return {
        "id": paper.get_short_id(),
        "title": paper.title,
        "authors": [author.name for author in paper.authors],
        "abstract": paper.summary,
        "categories": paper.categories,
        "published": paper.published.isoformat(),
        "url": paper.pdf_url,
        "resource_uri": f"arxiv://{paper.get_short_id()}",
    }

@mcp.tool()
async def search_papers(
    query: str, 
    max_results: int = 10, 
    date_from: str = None, 
    date_to: str = None, 
    categories: List[str] = None
) -> List[Dict[str, Any]]:
    """Search for papers on arXiv with advanced filtering.
    
    Args:
        query: Search query
        max_results: Maximum number of results to return
        date_from: Search start date (YYYY-MM-DD)
        date_to: Search end date (YYYY-MM-DD)
        categories: List of category filters
        
    Returns:
        List of searched papers
    """
    try:
        client = arxiv.Client()
        max_results = min(int(max_results), MAX_RESULTS)

        # Build search query with category filtering
        if categories:
            category_filter = " OR ".join(f"cat:{cat}" for cat in categories)
            query = f"({query}) AND ({category_filter})"

        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate,
        )

        # Process results with date filtering
        results = []
        try:
            date_from_obj = parser.parse(date_from).replace(tzinfo=timezone.utc) if date_from else None
            date_to_obj = parser.parse(date_to).replace(tzinfo=timezone.utc) if date_to else None
        except (ValueError, TypeError) as e:
            return [{"error": f"Invalid date format - {str(e)}"}]

        for paper in client.results(search):
            if _is_within_date_range(paper.published, date_from_obj, date_to_obj):
                results.append(_process_paper(paper))

            if len(results) >= max_results:
                break

        return results

    except Exception as e:
        logger.error(f"Search error: {str(e)}")
        return [{"error": f"Search failed: {str(e)}"}]

@mcp.tool()
async def download_paper(paper_id: str) -> Dict[str, Any]:
    """Download a paper from arXiv.
    
    Args:
        paper_id: arXiv paper ID
        
    Returns:
        Download result information
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(id_list=[paper_id])
        
        for paper in client.results(search):
            return {
                "id": paper.get_short_id(),
                "title": paper.title,
                "url": paper.pdf_url,
                "download_status": "success",
                "resource_uri": f"arxiv://{paper.get_short_id()}"
            }
        
        return {"error": f"Paper with ID {paper_id} not found"}
    except Exception as e:
        logger.error(f"Download error: {str(e)}")
        return {"error": f"Download failed: {str(e)}"}

@mcp.tool()
async def read_paper(paper_id: str) -> Dict[str, Any]:
    """Read the content of an arXiv paper.
    
    Args:
        paper_id: arXiv paper ID
        
    Returns:
        Paper content and metadata
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(id_list=[paper_id])
        
        for paper in client.results(search):
            return {
                "id": paper.get_short_id(),
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "abstract": paper.summary,
                "categories": paper.categories,
                "published": paper.published.isoformat(),
                "content_type": "text",
                "content": paper.summary  # Only providing the summary here, additional work needed to get full content
            }
        
        return {"error": f"Paper with ID {paper_id} not found"}
    except Exception as e:
        logger.error(f"Read error: {str(e)}")
        return {"error": f"Read failed: {str(e)}"}

@mcp.tool()
async def list_papers(category: str = None, max_results: int = 10) -> List[Dict[str, Any]]:
    """Get a list of the latest papers in a specific category.
    
    Args:
        category: arXiv category code
        max_results: Maximum number of results to return
        
    Returns:
        List of papers
    """
    try:
        client = arxiv.Client()
        max_results = min(int(max_results), MAX_RESULTS)
        
        query = f"cat:{category}" if category else ""
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate,
        )
        
        results = []
        for paper in client.results(search):
            results.append(_process_paper(paper))
            if len(results) >= max_results:
                break
                
        return results
    except Exception as e:
        logger.error(f"List error: {str(e)}")
        return [{"error": f"List failed: {str(e)}"}]

if __name__ == "__main__":
    mcp.run()

Overwriting mcp_server_arxiv_demo.py


### 2.2 ChEMBL MCP Server

In [3]:
%%writefile mcp_server_chembl_demo.py
from mcp.server.fastmcp import FastMCP
import logging
import sys
from typing import Any, List, Dict
from chembl_webresource_client.new_client import new_client

MAXIMUM_ACTIVITY = 100

logging.basicConfig(
    level=logging.INFO,
    format='%(filename)s:%(lineno)d | %(message)s',
    handlers=[
        logging.StreamHandler(sys.stderr)
    ]
)
logger = logging.getLogger("chembl_mcp")

try:
    mcp = FastMCP(
        name="chembl_tools",
    )
    logger.info("ChEMBL MCP server initialized successfully")
except Exception as e:
    err_msg = f"Error: {str(e)}"
    logger.error(f"{err_msg}")

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

@mcp.tool()
async def compount_activity(compound_name: str) -> List[Dict[str, Any]]:
    """activity data for the specified compound
    
    Args:
        compound_name: name of compound
        
    Returns:
        List of activity data
    """
    client = new_client
    molecule_id = client.molecule.filter(pref_name__iexact=compound_name).only('molecule_chembl_id')[0]
    # TODO: consider other types of activities
    activity = list(client.activity.filter(molecule_chembl_id=molecule_id['molecule_chembl_id']).filter(standard_type="IC50").only(['pchembl_value', 'assay_description', 'canonical_smiles']))
    if len(activity)>MAXIMUM_ACTIVITY:
        activity=activity[:MAXIMUM_ACTIVITY] # TODO: consider longer context
    return activity

@mcp.tool()
async def target_activity(target_name: str) -> List[Dict[str, Any]]:
    """activity data for the specified target
    
    Args:
        target_name: name of target
        
    Returns:
        List of activity data
    """
    client = new_client
    target_id = client.target.filter(target_synonym__icontains=target_name, organism='Homo sapiens').only('target_chembl_id')[0]
    # TODO: consider other types of activities
    activity = list(client.activity.filter(target_chembl_id=target_id['target_chembl_id']).filter(standard_type="IC50").only(['pchembl_value', 'assay_description', 'canonical_smiles']))
    if len(activity)>MAXIMUM_ACTIVITY:
        activity=activity[:MAXIMUM_ACTIVITY] # TODO: consider longer context
    return activity

if __name__ == "__main__":
    mcp.run()

Overwriting mcp_server_chembl_demo.py


## 3. Bedrock Model 설정

In [4]:
def get_bedrock_model(model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0"):
    """Amazon Bedrock 모델을 설정합니다."""
    return BedrockModel(
        boto_client_config=Config(
            read_timeout=600,
            connect_timeout=600,
            retries=dict(max_attempts=3, mode="adaptive"),
        ),
        model_id=model_id,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.9,
    )

# 모델 초기화
model = get_bedrock_model()
print(f"✓ Bedrock 모델 초기화 완료")

2025-09-15 03:45:57,110 - botocore.credentials - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


✓ Bedrock 모델 초기화 완료


## 4. 실습: HER2 관련 연구 조사

HER2 (Human Epidermal Growth Factor Receptor 2)는 유방암 치료의 중요한 표적입니다.

### 4.1 예제 1: HER2 관련 최신 논문 조사

In [5]:
question_1 = "HER2 관련 최신 논문을 조사해줘."

print("🔍 질문:", question_1)
print("="*80)

🔍 질문: HER2 관련 최신 논문을 조사해줘.


In [6]:
# 기본 Agent 생성
def create_basic_agent():
    """MCP 도구 없이 기본 Agent를 생성합니다."""
    system_prompt = """
    당신은 생명과학 분야의 전문 연구원입니다. 
    사용자의 질문에 대해 정확하고 과학적인 답변을 제공해야 합니다.
    
    다만, 외부 데이터베이스에 직접 접근할 수는 없으므로,
    일반적인 지식을 바탕으로 답변하세요.
    """
    
    return Agent(
        model=model,
        system_prompt=system_prompt,
        tools=[],  # 도구 없음
    )

basic_agent = create_basic_agent()
print("✓ 기본 Agent 생성 완료")

# 기본 Agent 응답
print("\n📚 기본 Agent 응답 (MCP 없음):")
print("-"*40)

response_basic = basic_agent(question_1)
print(response_basic)

2025-09-15 03:45:57,172 - strands.telemetry.metrics - INFO - Creating Strands MetricsClient


✓ 기본 Agent 생성 완료

📚 기본 Agent 응답 (MCP 없음):
----------------------------------------
# HER2 관련 최신 연구 동향

안녕하세요. 생명과학 연구원입니다. HER2 관련 최신 연구 동향에 대해 일반적인 지식을 바탕으로 안내해 드리겠습니다.

## 최근 HER2 연구 주요 방향

최근 HER2 연구는 다음과 같은 주요 영역에서 활발히 진행되고 있습니다:

1. **항체-약물 접합체(ADC) 개발**
   - Trastuzumab deruxtecan(T-DXd)과 같은 차세대 ADC의 효능 및 내성 기전 연구
   - HER2-low 발현 종양에서의 ADC 효과 평가

2. **면역치료와의 병용요법**
   - 면역관문억제제와 HER2 표적치료제의 병용 효과
   - HER2 CAR-T 세포 치료법의 개선 및 임상 적용

3. **HER2 돌연변이 및 이형성**
   - 다양한 암종에서 HER2 돌연변이의 임상적 의미
   - 종양 내 HER2 발현 이질성과 치료 반응 관계

4. **내성 기전 극복 전략**
   - HER2 표적치료 내성 기전 규명
   - PI3K/AKT/mTOR 경로 억제제와의 병용 전략

## 참고사항

실시간 최신 논문 정보를 제공하기 위해서는 PubMed, Science Direct, Nature 등의 학술 데이터베이스를 직접 검색하시는 것이 가장 정확합니다. 특정 논문에 대한 자세한 정보가 필요하시면 말씀해 주세요.# HER2 관련 최신 연구 동향

안녕하세요. 생명과학 연구원입니다. HER2 관련 최신 연구 동향에 대해 일반적인 지식을 바탕으로 안내해 드리겠습니다.

## 최근 HER2 연구 주요 방향

최근 HER2 연구는 다음과 같은 주요 영역에서 활발히 진행되고 있습니다:

1. **항체-약물 접합체(ADC) 개발**
   - Trastuzumab deruxtecan(T-DXd)과 같은 차세대 ADC의 효능 및 내성 기전 연구
   - HER2-low 발현 종양에

In [7]:
# MCP Agent 응답
print("\n🔬 MCP Agent 응답 (Arxiv 검색 포함):")
print("-"*40)

# MCP Agent 생성 및 실행
try:
    # MCP 클라이언트와 함께 Agent 실행
    arxiv_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_arxiv_demo.py"])
    ))
    
    with arxiv_client as client:
        tool = client.list_tools_sync()
        
        mcp_agent = Agent(
            model=model,
            system_prompt="""
            당신은 생명과학 연구원입니다. Arxiv를 검색하여 최신 논문 정보를 제공할 수 있습니다.
            검색 결과를 바탕으로 구체적인 논문 제목, 저자, 주요 발견사항을 포함하여 답변하세요.
            """,
            tools=[tool]
        )
        
        response_mcp = mcp_agent(question_1)
        print(response_mcp)
        
except Exception as e:
    print(f"오류 발생: {e}")
    print("MCP 서버가 실행 중인지 확인하세요.")


🔬 MCP Agent 응답 (Arxiv 검색 포함):
----------------------------------------
HER2(Human Epidermal growth factor Receptor 2) 관련 최신 논문을 조사해 드리겠습니다. 생명과학 분야에서 HER2는 특히 유방암 및 기타 암 연구에서 중요한 표적입니다. 최신 연구 동향을 확인하기 위해 arXiv에서 검색해 보겠습니다.
Tool #1: search_papers
HER2에 더 집중된 논문을 찾기 위해 검색어를 좀 더 구체적으로 해보겠습니다.
Tool #2: search_papers
HER2에 더 집중된 검색을 위해 마지막으로 검색어를 더 구체화해보겠습니다.
Tool #3: search_papers
HER2에 더 집중된 검색을 위해 마지막으로 검색어를 더 구체화하고 최신 논문을 찾아보겠습니다.
Tool #4: search_papers
HER2에 더 집중된 검색을 위해 마지막으로 검색어를 더 구체화하고 생물학 분야로 제한해보겠습니다.
Tool #5: search_papers
생물학 분야에서 더 넓은 범위로 검색해보겠습니다.
Tool #6: search_papers
검색 결과를 살펴본 결과, HER2 양성 유방암에 관한 최신 연구 논문을 찾기 위해 PubMed나 다른 의학 데이터베이스를 활용하는 것이 더 적절할 것 같습니다. arXiv에서는 HER2 관련 최신 연구가 많이 발견되지 않았습니다. 그럼에도 불구하고, 검색 결과를 바탕으로 유방암 관련 최신 연구 동향을 정리해 드리겠습니다.

# HER2 관련 최신 연구 동향

HER2(Human Epidermal growth factor Receptor 2)는 유방암 및 기타 암종에서 중요한 표적으로, 특히 유방암 환자의 약 15-20%에서 과발현되는 것으로 알려져 있습니다. 현재 arXiv에서 검색된 최신 연구 동향을 바탕으로 유방암 및 관련 연구에 대해 정리해 드리겠습니다.

## 1. 유방암 진단 기술의 발전

**GLAM: Geometr

### 4.2 예제 2: HER2 관련 화합물 검색

In [8]:
question_2 = "HER2 관련 화합물을 검색해줘."

print("🔍 질문:", question_2)
print("="*80)

🔍 질문: HER2 관련 화합물을 검색해줘.


In [9]:
# 기본 Agent 응답
print("\n📚 기본 Agent 응답 (MCP 없음):")
print("-"*40)

response_basic_2 = basic_agent(question_2)
print(response_basic_2)


📚 기본 Agent 응답 (MCP 없음):
----------------------------------------
# HER2 관련 주요 화합물 정보

HER2(Human Epidermal Growth Factor Receptor 2) 관련 주요 화합물들을 아래와 같이 정리해 드립니다.

## 승인된 HER2 표적 치료제

### 단클론항체
- **트라스투주맙(Trastuzumab)**: 최초의 HER2 표적 항체, HER2 세포외 도메인에 결합
- **퍼투주맙(Pertuzumab)**: HER2의 이량체화 도메인에 결합하여 HER2-HER3 이량체 형성 억제

### 항체-약물 접합체(ADC)
- **T-DM1(Trastuzumab emtansine)**: 트라스투주맙에 세포독성 물질 DM1 결합
- **T-DXd(Trastuzumab deruxtecan)**: 트라스투주맙에 토포이소머라제 I 억제제 결합
- **SYD985(Trastuzumab duocarmazine)**: 트라스투주맙에 duocarmycin 유도체 결합

### 티로신 키나아제 억제제(TKI)
- **라파티닙(Lapatinib)**: HER2와 EGFR의 세포내 티로신 키나아제 도메인 이중 억제제
- **네라티닙(Neratinib)**: 비가역적 범-HER 억제제
- **투카티닙(Tucatinib)**: 선택적 HER2 억제제, 뇌 혈관장벽 투과성 우수

## 개발 중인 화합물

### 새로운 ADC
- **ARX788**: 사이트 특이적 결합 기술로 개발된 HER2 ADC
- **RC48(Disitamab vedotin)**: 새로운 HER2 항체에 MMAE 결합

### 이중특이성 항체
- **ZW25(Zanidatamab)**: 두 개의 다른 HER2 에피토프 표적
- **MCLA-128(Zenocutuzumab)**: HER2/HER3 이중특이성 항체

### 기타 접근법
- **마르게툭시맙(Margetuximab)**: Fc 도메인 최적화된 HER2 항체
- **포지오티닙(P

In [10]:
# MCP Agent 응답
print("\n💊 MCP Agent 응답 (ChEMBL 검색 포함):")
print("-"*40)

# MCP Agent 생성 및 실행
try:
    # ChEMBL 클라이언트와 함께 Agent 실행
    chembl_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_chembl_demo.py"])
    ))
    
    with chembl_client as client:
        tool = client.list_tools_sync()
        
        mcp_agent = Agent(
            model=model,
            system_prompt="""
            당신은 약물 발견 연구원입니다. ChEMBL 데이터베이스를 검색하여 화합물 정보를 제공할 수 있습니다.
            검색 결과를 바탕으로 화합물의 ChEMBL ID, 활성 값, SMILES 구조 등을 포함하여 답변하세요.
            """,
            tools=[tool]
        )
        
        response_mcp_2 = mcp_agent(question_2)
        print(response_mcp_2)
        
except Exception as e:
    print(f"오류 발생: {e}")
    print("MCP 서버가 실행 중인지 확인하세요.")


💊 MCP Agent 응답 (ChEMBL 검색 포함):
----------------------------------------
HER2(Human Epidermal growth factor Receptor 2) 관련 화합물에 대한 정보를 검색해드리겠습니다. HER2는 유방암 등 여러 암종에서 중요한 표적이 되는 단백질입니다. HER2 표적 활성 데이터를 확인해보겠습니다.
Tool #1: target_activity
HER2 관련 화합물 검색 결과를 분석해드리겠습니다. 검색 결과에서 다양한 HER2 억제제 화합물들이 확인되었습니다.

### 주요 HER2 억제제 화합물 정보

1. **가장 활성이 높은 화합물들**:
   - pChEMBL 값이 8.0 이상인 강력한 HER2 억제제:
     - `CS(=O)(=O)CCNc1nc(-c2ccc3ncnc(Nc4ccc(OCc5cccc(F)c5)c(Cl)c4)c3c2)cs1` (pChEMBL: 8.05, IC50: 0.009 μM)
     - `O=C(/C=C/C(=O)Nc1cc2c(Nc3cccc(Br)c3)ncnc2cn1)NCCCn1ccnc1` (pChEMBL: 8.09, IC50: 8.1 nM)
     - `O=C(/C=C/C(=O)Nc1cc2c(Nc3cccc(Br)c3)ncnc2cn1)NCCCN1CCOCC1` (pChEMBL: 8.30, IC50: 5.0 nM)

2. **퀴나졸린 기반 화합물**:
   - 많은 HER2 억제제가 퀴나졸린(quinazoline) 구조를 기반으로 하고 있습니다:
     - `COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1` (pChEMBL: 6.62, IC50: 0.24 μM)
     - `C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1` (pChEMBL: 6.12, IC50: 0.76 μM)

3. **피롤 기반 화합물**:
   - 피롤 구조를 가진 HER2 억제제:
     - `Cc1cc(C(=O)NC

## 5. 통합 Agent: Arxiv + ChEMBL

두 데이터베이스를 모두 활용하는 통합 Agent를 만들어 더 포괄적인 연구를 수행합니다.

In [11]:
def create_integrated_agent():
    """Arxiv와 ChEMBL을 모두 활용하는 통합 Agent를 생성합니다."""
    
    system_prompt = """
    당신은 약물 발견 분야의 수석 연구원입니다.
    다음 데이터베이스에 접근할 수 있습니다:
    
    1. Arxiv: 최신 과학 논문과 연구 동향
    2. ChEMBL: 화합물, 타겟, 생물학적 활성 데이터
    
    연구 질문에 답할 때:
    - 먼저 Arxiv에서 최신 연구 동향을 파악하세요
    - ChEMBL에서 관련 화합물과 활성 데이터를 검색하세요
    - 두 소스의 정보를 통합하여 포괄적인 답변을 제공하세요
    - 항상 출처를 명시하고 데이터를 구체적으로 제시하세요
    """
    
    tools = []
    
    # Arxiv 도구 로드
    arxiv_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_arxiv_demo.py"])
    ))
    
    # ChEMBL 도구 로드
    chembl_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_chembl_demo.py"])
    ))
    
    # 두 클라이언트를 동시에 사용
    with arxiv_client as ax_client, chembl_client as ch_client:
        arxiv_tools = ax_client.list_tools_sync()
        chembl_tools = ch_client.list_tools_sync()
        
        tools.extend(arxiv_tools)
        tools.extend(chembl_tools)
        
        print(f"✓ 총 {len(tools)}개 도구 로드 완료")
        print(f"  - Arxiv: {len(arxiv_tools)}개")
        print(f"  - ChEMBL: {len(chembl_tools)}개")
        
        return Agent(
            model=model,
            system_prompt=system_prompt,
            tools=tools,
        ), arxiv_client, chembl_client

print("✓ 통합 Agent 함수 준비 완료")

✓ 통합 Agent 함수 준비 완료


### 5.1 통합 연구 질문

In [12]:
integrated_question = """
HER2 양성 유방암 치료에 대한 종합적인 분석을 수행해줘:
1. 최근 발표된 주요 연구 논문들을 찾아줘
2. HER2를 타겟으로 하는 주요 화합물들과 그들의 활성 데이터를 조사해줘
"""

print("🔬 통합 연구 질문:")
print(integrated_question)
print("="*80)

🔬 통합 연구 질문:

HER2 양성 유방암 치료에 대한 종합적인 분석을 수행해줘:
1. 최근 발표된 주요 연구 논문들을 찾아줘
2. HER2를 타겟으로 하는 주요 화합물들과 그들의 활성 데이터를 조사해줘



In [ ]:
# 통합 Agent 실행
print("\n🚀 통합 Agent 실행 (Arxiv + ChEMBL):")
print("-"*40)

try:
    # 통합 Agent와 클라이언트 생성
    arxiv_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_arxiv_demo.py"])
    ))
    
    chembl_client = MCPClient(lambda: stdio_client(
        StdioServerParameters(command="python", args=["mcp_server_chembl_demo.py"])
    ))
    
    with arxiv_client as ax_client, chembl_client as ch_client:
        # 도구 수집
        tools = []
        tools.extend(ax_client.list_tools_sync())
        tools.extend(ch_client.list_tools_sync())
        
        # Agent 생성
        integrated_agent = Agent(
            model=model,
            system_prompt="""
            당신은 약물 발견 분야의 수석 연구원입니다.
            Arxiv와 ChEMBL 데이터베이스를 활용하여 포괄적인 연구 분석을 수행하세요.
            각 데이터베이스에서 얻은 정보를 명확히 구분하여 제시하고,
            과학적 근거와 함께 통합적인 인사이트를 제공하세요.
            """,
            tools=tools
        )
        
        # 질문 실행
        response = integrated_agent(integrated_question)
        print(response)
        
except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()


🚀 통합 Agent 실행 (Arxiv + ChEMBL):
----------------------------------------
HER2 양성 유방암 치료에 대한 종합적인 분석을 수행하겠습니다. 먼저 arXiv에서 최근 발표된 관련 연구 논문들을 검색하고, 이어서 ChEMBL 데이터베이스에서 HER2를 타겟으로 하는 화합물들의 활성 데이터를 조사하겠습니다.

## 1. 최근 발표된 HER2 양성 유방암 관련 주요 연구 논문
Tool #1: search_papers


## 6. 성능 비교 및 분석

In [ ]:
# 비교 분석을 위한 함수
def compare_agents_responses(question):
    """기본 Agent와 MCP Agent의 응답을 비교합니다."""
    
    results = {
        "question": question,
        "basic_agent": None,
        "mcp_agent": None,
        "analysis": None
    }
    
    # 기본 Agent 응답
    print("🤖 기본 Agent 처리 중...")
    basic_response = basic_agent(question)
    results["basic_agent"] = basic_response
    
    # MCP Agent 응답
    print("🔬 MCP Agent 처리 중...")
    try:
        arxiv_client = MCPClient(lambda: stdio_client(
            StdioServerParameters(command="python", args=["mcp_server_arxiv_demo.py"])
        ))
        chembl_client = MCPClient(lambda: stdio_client(
            StdioServerParameters(command="python", args=["mcp_server_chembl_demo.py"])
        ))
        
        with arxiv_client as ax_client, chembl_client as ch_client:
            tools = []
            tools.extend(ax_client.list_tools_sync())
            tools.extend(ch_client.list_tools_sync())
            
            mcp_agent = Agent(
                model=model,
                system_prompt="외부 데이터베이스를 활용하여 정확한 정보를 제공하는 연구원",
                tools=tools
            )
            
            mcp_response = mcp_agent(question)
            results["mcp_agent"] = mcp_response
            
    except Exception as e:
        results["mcp_agent"] = f"오류: {str(e)}"
    
    # 분석
    analysis = """
    주요 차이점:
    1. 데이터 최신성: MCP Agent는 실시간 데이터베이스 검색 가능
    2. 구체성: MCP Agent는 실제 논문 ID, 화합물 ID 등 구체적 정보 제공
    3. 검증 가능성: MCP Agent의 답변은 출처 확인 가능
    4. 포괄성: MCP Agent는 여러 데이터베이스 통합 검색 가능
    """
    results["analysis"] = analysis
    
    return results

print("✓ 비교 분석 함수 준비 완료")

## 7. 정리

이 노트북에서 학습한 내용:

✅ MCP (Model Context Protocol)를 통한 외부 데이터베이스 연동
✅ Arxiv와 ChEMBL 데이터베이스를 Agent의 도구로 활용
✅ 기본 Agent와 MCP 활용 Agent의 성능 차이 비교
✅ 실제 생명과학 연구 질문에 대한 응답 생성

### 핵심 포인트

1. **MCP의 장점**: 실시간 데이터 접근, 검증 가능한 정보, 다양한 소스 통합
2. **활용 분야**: 약물 발견, 문헌 조사, 타겟 검증, 화합물 스크리닝
3. **확장 가능성**: 더 많은 데이터베이스와 도구 추가 가능

다음 노트북에서는 내부 데이터베이스와의 연동을 다룰 예정입니다.